## 心理實習排課腳本

### 1. 設定組態

此區塊程式碼用於設定整個排課流程中會使用到的全域變數和參數。
這包括了:
- 輸入檔案的路徑
- 轉換後的資料欄位的定義 (運算中間表格)
- 轉換後的元數據欄位 (運算中間表格)
- 課程名稱列表 (運算中間表格)
- 排課的週數
- 每週對應的原始資料欄位
- 時間區段的字串定義
- 課程和學生的限制條件（如課程最大容量、學生每週可選課數）

In [ ]:
# config
file_path = 'data.csv'
columns = [
    'name',
    'student_id',
    '周一上午',
    '周一下午',
    '周二上午',
    '周二下午',
    '周三上午',
    '周三下午',
    '周四上午',
    '周四下午',
    '周五上午',
    '周五下午'
]
meta_data_col = [
    2, 3
]
class_name_list = columns[2:]

week_cnt = 7
each_week_col = [
    # [5, 6, 7, 8, 9],
    [10, 11, 12, 13, 14],
    [15, 16, 17, 18, 19],
    [20, 21, 22, 23, 24],
    [25, 26, 27, 28, 29],
    [30, 31, 32, 33, 34],
    [35, 36, 37, 38, 39],
    [40, 41, 42, 43, 44],
]
timeStrDef = [
    '9:00~12:00',
    '14:00~17:00'
]

class_max_cap = 8
student_week_capacity = 1


### 2. 讀取與預處理原始資料

這個區塊使用 pandas 函式庫來讀取指定的 CSV 檔案 (data.csv)。接著，它會對資料進行初步的清理和整理，包括：

- 根據提交時間（第一欄）進行排序。
- 根據學號移除重複的報名資料，只保留最新的一筆。

最後，印出前五筆資料以供檢視。

In [ ]:
import pandas as pd

source_df = pd.read_csv(file_path)
source_df.sort_values(by=[source_df.columns[0]], ascending=True, inplace=True)
source_df.drop_duplicates(subset=['學號'], keep='last', inplace=True, ignore_index=False)
print(source_df.head()) # This will show the first 5 rows of your data

### 3. 初始化每週的資料結構

此區塊程式碼根據設定的週數 (week_cnt)，建立一個陣列 (df_array)

其中每個元素都是一個空的 pandas DataFrame。

這些 DataFrame 將用於後續步驟中，分別儲存每一週的學生選課狀態。

每個 DataFrame 都擁有預先定義好的欄位結構。

關鍵是每一周是一次上課循環，要依據學生選課在每一週排出實際修課表。

In [ ]:
import pandas as pd

df_array = [pd.DataFrame(columns=columns) for _ in range(week_cnt)]

print(f"陣列的長度為：{len(df_array)}")
print("\n第一個 DataFrame：")
print(df_array[0])


### 4. 資料轉換與整理

這個區塊的目的是將從原始 CSV 讀取的資料，轉換為後續演算法所需的格式。

它會遍歷每一週的設定，並為每個學生處理其所選擇的時段。

具體來說，它會將原始表格中表示時間區段的文字（例如 '9:00~12:00'），轉換為布林值（某日上午課程 True 或 False），

以表示該學生在特定時段是否有空。

轉換後的資料會被分別填入先前為每一週建立的 DataFrame 中。

In [ ]:
week_idx = 0
for cols in each_week_col:
    target_col = meta_data_col + cols
    selected_df = source_df.iloc[:, target_col]

    for index, row in selected_df.iterrows():
            tmp_data =  []
            for i in range(len(meta_data_col)):
                tmp_data.append(row.iloc[i])
            for r in row[len(meta_data_col):]:
                for i in timeStrDef:
                    if i in str(r):
                        tmp_data.append(True)
                    else:
                        tmp_data.append(False)
            tmp_df = pd.DataFrame([
                tmp_data,
            ], columns=columns)
            df_array[week_idx] = pd.concat([df_array[week_idx], tmp_df], ignore_index=True)
    week_idx = week_idx + 1

print(df_array[0])

### 5. 核心排課演算法：最大流模型

此區塊是整個排課系統的核心。

它為每一週的資料建立一個網路流模型（Network Flow Model），並使用「最大流最小割定理」（Max-Flow Min-Cut Theorem）來找出最佳的學生與課程時段分配方案。

建立有向圖

- 建立圖 (Graph)：為每一週，程式會建立一個有向圖，包含：
- 一個源點 (source) 和一個匯點 (sink)。
- 代表每位學生的節點。
- 代表每個課程時段的節點。

設定容量 (Capacity)：

- 從源點到每個學生的邊，容量為學生每週可上的課堂數 (希望學生選一堂，因此都設置為 1)。
- 從學生到其可選時段的邊，容量為 1。(學生一個時段只能上一門課)
- 從課程時段到匯點的邊，容量為該時段的學生上限。(每堂課最多可以承受幾個人)

分析結果

- 求解最大流：透過 networkx 函式庫計算從源點到匯點的最大流量。一個成功的分配意味著所有學生都被安排了課程（即從源點流出的流量等於學生總數）。
- 結果分析：最後，程式會分析流量分配的結果，判斷該週的排課是否成功，並將成功分配的學生名單儲存起來。

In [ ]:
import networkx as nx

w_ret = {}
w_idx = 1
for w_arr in df_array:
    # 建立一個有向圖
    G = nx.DiGraph()

    source = "students"
    sink = "class"
    G.add_node(source)
    G.add_node(sink)

    for col_name in class_name_list:
        class_node_name = col_name
        G.add_node(class_node_name)
        G.add_edge(class_node_name, sink, capacity=class_max_cap)

    for index, row in w_arr.iterrows():
        student_node_name = row['name']+'_'+str(row['student_id'])
        G.add_node(student_node_name)
        G.add_edge(source, student_node_name, capacity=student_week_capacity)
        for classes_name in class_name_list:
            if row[classes_name]:
                G.add_edge(student_node_name, classes_name, capacity=1)


    flow_value, flow_dict = nx.maximum_flow(G, source, sink)

    # print(f"最大流量值：{flow_value}")

    # 印出每條邊的流量分配
    # print("各邊的流量：")
    # tt = 1
    is_all_student_have_class = True
    empty_student_list = []
    student_list = []
    for u, v_dict in flow_dict.items():
        for v, flow in v_dict.items():
            if u == source and flow == 0:
                # print(f" {tt}  從 {u} 到 {v} 的流量為 {flow}")
                # tt = tt+1
                is_all_student_have_class = False
                empty_student_list.append(v)
            if u != source and v != sink and flow == 1:
                student_list.append((u, v))

    if is_all_student_have_class:
        print(f"week {w_idx+1} 成功分配")
        # print(len(student_list), student_list)
        w_ret[w_idx] = student_list
    else:
        print(f"week {w_idx+1} 成功失敗")
        print(empty_student_list)
    w_idx = w_idx + 1

# print(w_ret[1])

### 6. 匯出排課結果

最後，這個區塊程式碼會將前一步驟中成功分配的排課結果，匯出成 CSV 檔案。

它會遍歷儲存著成功結果的 w_ret 字典，為每一週建立一個獨立的 CSV 檔案（例如 week_1_output.csv, week_2_output.csv 等）

檔案中包含兩欄：「姓名_學號」和該學生被分配到的「時段」。

In [ ]:
import pandas as pd

for w in range(week_cnt+1):
    # 您的原始資料
    if w == 0:
        continue
    data = w_ret[w]

    # 將資料轉換為 DataFrame，並指定欄位名稱
    df = pd.DataFrame(data, columns=['姓名_學號', '時段'])

    # 將 DataFrame 匯出為 CSV 檔案
    # index=False 避免將索引寫入 CSV 檔案
    output_filename = f"week_{w}_output.csv"
    df.to_csv(output_filename, index=False, encoding='utf-8')

    print(f"資料已成功匯出至 {output_filename}")
